# 🤖 Understanding Transformers in LLMs - Interactive Notebook

This notebook will teach you how Transformers work - the architecture behind ChatGPT, Claude, and all modern LLMs!

## 📚 What You'll Learn

1. **Self-Attention** - How words "pay attention" to each other
2. **Multi-Head Attention** - Multiple perspectives in parallel
3. **Positional Encoding** - How transformers know word order
4. **Transformer Blocks** - Complete architecture
5. **Text Generation** - How LLMs predict next words

## 🚀 Let's Get Started!

Run each cell in order by pressing `Shift + Enter`

---
## Setup: Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Set random seed for reproducibility
np.random.seed(42)

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")

---
## 📝 Part 1: Word Embeddings

### Concept
In LLMs, words are converted to **vectors of numbers** (embeddings). These vectors capture the meaning of words in a continuous space.

**Example**: 
- "cat" → [0.5, 0.8, 0.2, 0.3]
- "dog" → [0.6, 0.7, 0.3, 0.2]  (similar to cat!)
- "car" → [0.1, 0.2, 0.9, 0.8]  (very different)

In [ ]:
# Our example sentence
sentence = "The cat sat on the mat"
words = sentence.split()

print(f"📝 Sentence: '{sentence}'")
print(f"🔢 Number of words: {len(words)}")
print(f"📋 Words: {words}")

# Create simple word embeddings (4-dimensional)
d_model = 64  # Embedding dimension (real LLMs use 768-12288)
seq_len = len(words)

# Random embeddings (in real LLMs, these are learned from data)
embeddings = np.random.randn(seq_len, d_model) * 0.5

print(f"\n✅ Created embeddings with shape: {embeddings.shape}")
print(f"   (sequence_length={seq_len}, embedding_dimension={d_model})")

# Show first word's embedding (first 10 values)
print(f"\n🔍 Example - '{words[0]}' embedding (first 10 values):")
print(f"   {embeddings[0, :10]}")

---
## 📍 Part 2: Positional Encoding

### Problem
Transformers process all words **in parallel** (not sequentially like RNNs). So how do they know word order?

**"The cat sat"** vs **"Sat cat the"** - Same words, different meanings!

### Solution
Add **position information** to embeddings using sine/cosine functions.

In [ ]:
def create_positional_encoding(seq_len, d_model):
    """
    Create positional encoding using sine and cosine functions.
    
    PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
    PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
    """
    position = np.arange(seq_len)[:, np.newaxis]
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
    
    pe = np.zeros((seq_len, d_model))
    pe[:, 0::2] = np.sin(position * div_term)  # Even indices
    pe[:, 1::2] = np.cos(position * div_term)  # Odd indices
    
    return pe

# Create positional encoding
pos_encoding = create_positional_encoding(seq_len, d_model)

print(f"✅ Created positional encoding: {pos_encoding.shape}")

# Add positional encoding to embeddings
embeddings_with_pos = embeddings + pos_encoding

print(f"\n📊 Before adding position:")
print(f"   Word 0 ('{words[0]}'): {embeddings[0, :5]}...")
print(f"\n📊 After adding position:")
print(f"   Word 0 ('{words[0]}'): {embeddings_with_pos[0, :5]}...")

print("\n💡 Now each word knows both its MEANING and its POSITION!")

### Visualize Positional Encoding

In [ ]:
# Create a larger positional encoding for visualization
pe_vis = create_positional_encoding(50, 128)

plt.figure(figsize=(12, 6))
plt.imshow(pe_vis.T, cmap='RdBu', aspect='auto')
plt.colorbar(label='Encoding Value')
plt.xlabel('Position in Sequence', fontsize=12)
plt.ylabel('Embedding Dimension', fontsize=12)
plt.title('Positional Encoding Pattern (Sine/Cosine Waves)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Notice the wave patterns! Each position gets a unique 'fingerprint'.")

---
## 🎯 Part 3: Self-Attention Mechanism

### The Heart of Transformers!

Self-attention allows each word to "look at" all other words and decide which ones are important.

### Key Concepts
- **Query (Q)**: "What am I looking for?"
- **Key (K)**: "What do I contain?"
- **Value (V)**: "What information do I provide?"

### Formula
```
Attention(Q, K, V) = softmax(Q·K^T / √d_k) · V
```

In [ ]:
def softmax(x):
    """Numerically stable softmax."""
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

def self_attention(X, d_model):
    """
    Compute self-attention.
    
    Args:
        X: Input embeddings (seq_len, d_model)
        d_model: Dimension of embeddings
    
    Returns:
        output: Attention output
        attention_weights: Attention matrix for visualization
    """
    # Create Q, K, V weight matrices (normally learned during training)
    W_q = np.random.randn(d_model, d_model) * 0.01
    W_k = np.random.randn(d_model, d_model) * 0.01
    W_v = np.random.randn(d_model, d_model) * 0.01
    
    # Calculate Q, K, V
    Q = X @ W_q
    K = X @ W_k
    V = X @ W_v
    
    # Calculate attention scores
    scores = (Q @ K.T) / np.sqrt(d_model)
    
    # Apply softmax to get attention weights
    attention_weights = softmax(scores)
    
    # Apply attention to values
    output = attention_weights @ V
    
    return output, attention_weights

# Apply self-attention
X = embeddings_with_pos
attention_output, attention_weights = self_attention(X, d_model)

print("✅ Self-Attention computed!")
print(f"\n📊 Input shape: {X.shape}")
print(f"📊 Output shape: {attention_output.shape}")
print(f"📊 Attention weights shape: {attention_weights.shape}")

print("\n💡 Each word now has a NEW representation that considers ALL other words!")

### Visualize Attention Matrix

In [ ]:
# Create attention heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(attention_weights, 
            annot=True, 
            fmt='.2f', 
            cmap='YlOrRd',
            xticklabels=words,
            yticklabels=words,
            cbar_kws={'label': 'Attention Weight'})
plt.xlabel('Key (Word being attended to)', fontsize=12)
plt.ylabel('Query (Word asking)', fontsize=12)
plt.title('Self-Attention Matrix: How Words Attend to Each Other', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n📊 How to read this matrix:")
print("   • Rows = Query words (asking)")
print("   • Columns = Key words (being asked about)")
print("   • Brighter colors = Stronger attention")
print("   • Each row sums to 1.0 (100% attention distributed)")

# Show which word each word attends to most
print("\n🔍 Attention patterns:")
for i, word in enumerate(words):
    max_idx = np.argmax(attention_weights[i])
    max_attn = attention_weights[i, max_idx]
    print(f"   '{word}' attends most to '{words[max_idx]}' ({max_attn:.1%})")

---
## 🧠 Part 4: Multi-Head Attention

### Why Multiple Heads?
Different attention heads can learn different types of relationships:
- **Head 1**: Subject-verb relationships
- **Head 2**: Adjective-noun relationships
- **Head 3**: Long-range dependencies
- etc.

It's like having **multiple experts** looking at the sentence from different angles!

In [ ]:
def multi_head_attention(X, d_model, num_heads):
    """
    Compute multi-head attention.
    
    Args:
        X: Input embeddings (seq_len, d_model)
        d_model: Dimension of embeddings
        num_heads: Number of attention heads
    
    Returns:
        output: Multi-head attention output
        all_attention_weights: List of attention weights from each head
    """
    assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
    
    d_k = d_model // num_heads
    seq_len = X.shape[0]
    
    all_attention_weights = []
    head_outputs = []
    
    # Process each head
    for head in range(num_heads):
        # Extract this head's portion of embeddings
        start_idx = head * d_k
        end_idx = (head + 1) * d_k
        X_head = X[:, start_idx:end_idx]
        
        # Compute attention for this head
        output, attn_weights = self_attention(X_head, d_k)
        
        head_outputs.append(output)
        all_attention_weights.append(attn_weights)
    
    # Concatenate all head outputs
    concat_output = np.concatenate(head_outputs, axis=-1)
    
    # Final linear projection
    W_o = np.random.randn(d_model, d_model) * 0.01
    output = concat_output @ W_o
    
    return output, all_attention_weights

# Apply multi-head attention
num_heads = 8
mha_output, all_attention = multi_head_attention(X, d_model, num_heads)

print(f"✅ Multi-Head Attention computed with {num_heads} heads!")
print(f"\n📊 Each head processes {d_model//num_heads} dimensions")
print(f"📊 Output shape: {mha_output.shape}")
print(f"📊 Number of attention matrices: {len(all_attention)}")

### Visualize Different Attention Heads

In [ ]:
# Visualize first 4 heads
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for i in range(4):
    sns.heatmap(all_attention[i], 
                annot=True, 
                fmt='.2f',
                cmap='viridis',
                xticklabels=words,
                yticklabels=words,
                ax=axes[i],
                cbar_kws={'label': 'Attention'})
    axes[i].set_title(f'Head {i+1} Attention Pattern', fontweight='bold')
    axes[i].set_xlabel('Key')
    axes[i].set_ylabel('Query')

plt.tight_layout()
plt.show()

print("\n💡 Notice how different heads learn different patterns!")
print("   Each head specializes in different types of relationships.")

---
## 🔄 Part 5: Feed-Forward Network

After attention, each position goes through a **feed-forward network**:

```
FFN(x) = max(0, xW₁ + b₁)W₂ + b₂
```

Typically: **d_model → 4×d_model → d_model**

This adds **non-linear transformations** to help the model learn complex patterns.

In [ ]:
def feed_forward_network(X, d_model, d_ff=None):
    """
    Apply feed-forward network to each position.
    
    Args:
        X: Input (seq_len, d_model)
        d_model: Input/output dimension
        d_ff: Hidden dimension (default: 4 * d_model)
    
    Returns:
        Output of feed-forward network
    """
    if d_ff is None:
        d_ff = 4 * d_model
    
    # First linear layer
    W1 = np.random.randn(d_model, d_ff) * 0.01
    b1 = np.zeros(d_ff)
    
    # Second linear layer
    W2 = np.random.randn(d_ff, d_model) * 0.01
    b2 = np.zeros(d_model)
    
    # Apply FFN with ReLU activation
    hidden = np.maximum(0, X @ W1 + b1)  # ReLU
    output = hidden @ W2 + b2
    
    return output

# Apply feed-forward network
d_ff = 4 * d_model
ffn_output = feed_forward_network(mha_output, d_model, d_ff)

print("✅ Feed-Forward Network applied!")
print(f"\n📊 Architecture: {d_model} → {d_ff} → {d_model}")
print(f"📊 Input shape: {mha_output.shape}")
print(f"📊 Output shape: {ffn_output.shape}")
print("\n💡 FFN adds non-linear transformations to each position independently.")

---
## 🏗️ Part 6: Complete Transformer Block

A complete transformer block combines everything with **residual connections** and **layer normalization**:

```
1. Multi-Head Attention
2. Add & Normalize (Residual)
3. Feed-Forward Network
4. Add & Normalize (Residual)
```

### Residual Connections
Instead of: `output = Layer(input)`  
We do: `output = input + Layer(input)`

This helps train **very deep networks**!

In [ ]:
def layer_norm(x, eps=1e-6):
    """
    Apply layer normalization.
    """
    mean = np.mean(x, axis=-1, keepdims=True)
    std = np.std(x, axis=-1, keepdims=True)
    return (x - mean) / (std + eps)

def transformer_block(X, d_model, num_heads, d_ff):
    """
    Complete transformer block.
    
    Args:
        X: Input embeddings
        d_model: Embedding dimension
        num_heads: Number of attention heads
        d_ff: Feed-forward hidden dimension
    
    Returns:
        output: Transformed embeddings
        attention_weights: Attention weights for visualization
    """
    # 1. Multi-Head Attention + Residual + Norm
    attn_output, attn_weights = multi_head_attention(X, d_model, num_heads)
    X = layer_norm(X + attn_output)  # Residual connection
    
    # 2. Feed-Forward Network + Residual + Norm
    ffn_output = feed_forward_network(X, d_model, d_ff)
    X = layer_norm(X + ffn_output)  # Residual connection
    
    return X, attn_weights

# Apply one transformer block
block_output, block_attention = transformer_block(X, d_model, num_heads, d_ff)

print("✅ Complete Transformer Block processed!")
print(f"\n📊 Configuration:")
print(f"   • Embedding dimension: {d_model}")
print(f"   • Number of heads: {num_heads}")
print(f"   • Feed-forward dimension: {d_ff}")
print(f"\n📊 Input shape: {X.shape}")
print(f"📊 Output shape: {block_output.shape}")
print("\n💡 This is ONE layer. Real LLMs stack 12-96+ of these!")

---
## 📚 Part 7: Stacking Multiple Layers

Real transformers stack many blocks to build deeper understanding:

- **Early layers**: Learn simple patterns (adjacent words)
- **Middle layers**: Learn syntax and grammar
- **Deep layers**: Learn semantics and reasoning

Example layer counts:
- BERT: 12-24 layers
- GPT-3: 96 layers
- GPT-4: ~120 layers

In [ ]:
def stack_transformer_layers(X, d_model, num_heads, d_ff, num_layers):
    """
    Stack multiple transformer blocks.
    """
    all_attention = []
    
    for layer in range(num_layers):
        X, attn = transformer_block(X, d_model, num_heads, d_ff)
        all_attention.append(attn)
        print(f"   Layer {layer+1}/{num_layers} processed ✓")
    
    return X, all_attention

# Stack 6 transformer layers
num_layers = 6
print(f"🏗️ Stacking {num_layers} transformer layers...\n")

final_output, layer_attentions = stack_transformer_layers(
    embeddings_with_pos, d_model, num_heads, d_ff, num_layers
)

print(f"\n✅ Complete transformer with {num_layers} layers!")
print(f"📊 Final output shape: {final_output.shape}")
print(f"\n💡 Each word now has a rich, context-aware representation!")

---
## 🎲 Part 8: Predicting Next Word

The final step: Convert transformer output to **predictions**!

1. Take the last word's representation
2. Project to vocabulary size
3. Apply softmax to get probabilities
4. Sample or pick most likely word

This is how LLMs **generate text**!

In [ ]:
# Define vocabulary
vocab = ["The", "cat", "dog", "sat", "ran", "on", "mat", ".", "and", "is", 
         "was", "a", "in", "to", "that", "of", "for", "with", "as", "by"]
vocab_size = len(vocab)

print(f"📖 Vocabulary size: {vocab_size} words")
print(f"📋 Vocabulary: {vocab[:10]}...\n")

# Project to vocabulary
W_vocab = np.random.randn(d_model, vocab_size) * 0.01
last_word_repr = final_output[-1:, :]  # Last word's representation

logits = last_word_repr @ W_vocab
probs = softmax(logits)

# Get top predictions
top_k = 10
top_indices = np.argsort(probs[0])[::-1][:top_k]

print("🎲 Top 10 predictions for next word:\n")
print(f"{'Rank':<6} {'Word':<10} {'Probability':<15} {'Bar':<30}")
print("─" * 65)

for rank, idx in enumerate(top_indices, 1):
    word = vocab[idx]
    prob = probs[0, idx]
    bar_length = int(prob * 100)
    bar = "█" * bar_length
    print(f"{rank:<6} {word:<10} {prob:>7.1%}        {bar}")

predicted_word = vocab[top_indices[0]]
print(f"\n✅ Most likely next word: '{predicted_word}' 🎯")

### Visualize Probability Distribution

In [ ]:
# Plot probability distribution
plt.figure(figsize=(12, 6))
plt.bar(range(top_k), probs[0, top_indices], color='steelblue', alpha=0.7)
plt.xticks(range(top_k), [vocab[i] for i in top_indices], rotation=45)
plt.xlabel('Word', fontsize=12)
plt.ylabel('Probability', fontsize=12)
plt.title('Next Word Probability Distribution (Top 10)', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 The model assigns probability to each possible next word!")

---
## 📊 Part 9: Model Size Analysis

Let's compare our tiny model to real LLMs!

In [ ]:
# Calculate approximate parameter count
def count_parameters(d_model, num_heads, d_ff, num_layers, vocab_size):
    """
    Roughly estimate transformer parameters.
    """
    # Embeddings
    embedding_params = vocab_size * d_model
    
    # Per layer:
    # - Attention: 4 projection matrices (Q, K, V, O)
    attention_params = 4 * (d_model * d_model)
    # - Feed-forward: 2 linear layers
    ffn_params = 2 * (d_model * d_ff)
    # - Layer norms (2 per layer)
    norm_params = 4 * d_model
    
    layer_params = attention_params + ffn_params + norm_params
    total_layer_params = layer_params * num_layers
    
    # Output projection
    output_params = d_model * vocab_size
    
    total = embedding_params + total_layer_params + output_params
    return total

# Our model
our_params = count_parameters(d_model, num_heads, d_ff, num_layers, vocab_size)

# Compare to real models
models = {
    "Our Tiny Model": our_params,
    "BERT-Base": 110_000_000,
    "GPT-2": 1_500_000_000,
    "GPT-3": 175_000_000_000,
    "GPT-4 (est.)": 1_800_000_000_000
}

print("🎯 Model Size Comparison\n")
print(f"{'Model':<20} {'Parameters':<20} {'Scale'}")
print("─" * 60)

for name, params in models.items():
    if params < 1_000_000:
        scale = f"{params:,}"
    elif params < 1_000_000_000:
        scale = f"{params/1_000_000:.1f}M"
    else:
        scale = f"{params/1_000_000_000:.1f}B"
    
    print(f"{name:<20} {params:<20,} {scale}")

print(f"\n💡 Our model is {models['GPT-3'] / our_params:,.0f}x smaller than GPT-3!")
print("   But it uses the EXACT SAME architecture! 🎉")

### Visualize Model Sizes

In [ ]:
# Create bar chart with log scale
model_names = list(models.keys())
param_counts = list(models.values())

plt.figure(figsize=(12, 6))
bars = plt.bar(model_names, param_counts, color=['green', 'blue', 'orange', 'red', 'purple'], alpha=0.7)
plt.yscale('log')
plt.ylabel('Number of Parameters (log scale)', fontsize=12)
plt.title('Transformer Model Size Comparison', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, params in zip(bars, param_counts):
    height = bar.get_height()
    if params < 1_000_000_000:
        label = f"{params/1_000_000:.1f}M"
    else:
        label = f"{params/1_000_000_000:.0f}B"
    plt.text(bar.get_x() + bar.get_width()/2., height*1.1,
             label, ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

---
## 🎓 Summary: Key Concepts

### What You Learned

1. **Word Embeddings** - Convert words to vectors
2. **Positional Encoding** - Add position information
3. **Self-Attention** - Words "look at" each other
4. **Multi-Head Attention** - Multiple perspectives
5. **Feed-Forward Networks** - Non-linear transformations
6. **Residual Connections** - Enable deep networks
7. **Layer Stacking** - Build complex understanding
8. **Text Generation** - Predict next words

### Why Transformers Are Revolutionary

✅ **Parallel Processing** - Fast training on GPUs  
✅ **Long-Range Dependencies** - Can see entire context  
✅ **Scalability** - Works with billions of parameters  
✅ **Transfer Learning** - Pre-train once, fine-tune for many tasks  
✅ **Emergent Abilities** - Show reasoning, few-shot learning, etc.

### This Architecture Powers

- **GPT** (OpenAI) - Text generation
- **Claude** (Anthropic) - Advanced reasoning
- **BERT** (Google) - Understanding
- **LLaMA** (Meta) - Open-source LLMs
- And many more!

---

## 🚀 Next Steps

1. **Experiment** - Modify parameters and see what changes
2. **Implement** - Try building your own transformer from scratch
3. **Read Papers** - "Attention Is All You Need" (original paper)
4. **Explore Libraries** - Hugging Face Transformers, PyTorch, TensorFlow
5. **Build Projects** - Text classification, generation, translation

---

## 🎉 Congratulations!

You now understand the architecture behind modern LLMs!

The transformer you just learned about is the **exact same architecture** used in:
- ChatGPT
- Claude (me!)
- All modern LLMs

The only difference? They're just **much bigger** with billions of parameters trained on trillions of words!

**Keep learning and building!** 🚀

---
## 🎯 Bonus: Interactive Experiments

Try modifying these parameters and re-run the cells to see what happens!

In [ ]:
# Experiment with different configurations!
print("🧪 Try changing these parameters:\n")
print("Current configuration:")
print(f"  • d_model = {d_model}  (try: 32, 128, 256)")
print(f"  • num_heads = {num_heads}  (try: 2, 4, 16)")
print(f"  • num_layers = {num_layers}  (try: 3, 12, 24)")
print(f"  • d_ff = {d_ff}  (try: d_model*2, d_model*8)")
print("\n💡 Go back to the beginning and change these values!")
print("   Then re-run all cells to see the effects.")